# Revised case normalization for USZ 2019 (30.04.2020)

Notes:
- no case can be match on AdmNo in the year 2019

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:

file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/USZ_2018-2019_20200730.xlsx'),
         'USZ',
         '2019',
         'Rückmeldung_USZ_2019_30.04.2020')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/USZ_2018-2019_20200730.xlsx', hospital_name_db='USZ', year='2019', sheet='Rückmeldung_USZ_2019_30.04.2020')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-11 10:11:13.242 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 154 cases for USZ 2019
2022-11-11 10:11:13.247 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:63 - TYPES:
unnamed: 0           string
case_id              object
patient_id           object
kkik                 string
gender               string
age_years             int64
bfs_code             string
duration_of_stay      int64
pflegetage neu       string
pccl                  int64
pccl neu             string
old_pd               string
primary_diagnosis    string
added_icds           string
removed_icds         string
added_chops          string
removed_chops        string
drg                  string
drg neu              string
cw alt               string
cw neu               string
cw-änderung          string
kommentar dtod       string
kommentar usz        string
case_id_norm         object
dtype: object
2022-11-11 10:11:13.258 | I

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,6130587,10709587,M,54,13,6130587,J4400,J4400,M200,3,E06A,[F058],[],[],[]
1,6171196,5863929,M,62,5,6171196,J068,J068,M800,3,D62C,"[I8028, C770, C787]",[],[],[]
2,6269659,10160906,W,74,36,6269659,R11,M8088,M100,3,G72C,[R64],[],[],[]
3,6329461,10156863,W,79,6,6329461,G3531,G3531,M100,3,B68C,[R471],[],[],[]
4,6143166,10424299,M,75,5,6143166,J690,J690,M100,3,E77E,[E43],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,6373573,4958322,W,76,4,6373573,S3618,C23,M200,2,H64B,"[K650, C786]",[],[],[]
150,6475384,7878907,M,64,7,6475384,T8902,A490,M100,2,X60C,"[B9590, B957, D508]",[],[],[]
151,6413158,10274171,M,82,17,6413158,D352,D352,M200,2,B20A,"[I6201, I653]",[],[],[]
152,6351575,1422600,M,60,10,6351575,I730,I730,M100,2,I66B,"[L97, B965]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-11 10:11:16.395 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 14430 rows from the DB, for the hospital 'USZ' in 2019
2022-11-11 10:11:16.411 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:47 - 154 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


KeyError: "None of [Index(['revision_id', 'code'], dtype='object')] are in the [columns]"

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)